In [ ]:
import os
from skimage.morphology import remove_small_holes, remove_small_objects, binary_dilation, binary_erosion
from skimage.measure import regionprops, find_contours
from skimage.segmentation import active_contour
from scipy.ndimage import binary_dilation, distance_transform_edt, binary_fill_holes
from scipy.spatial.distance import cdist
import openslide
import matplotlib.pyplot as plt
import numpy as np
import glob
from pptx import Presentation
from pptx.util import Inches
from io import BytesIO
from PIL import Image, ImageDraw
from tqdm import tqdm
import cv2

In [ ]:
def closest_farthest_points(origin, point_set):
    """Calculate the closest and furthest point in a set from a specified origin.

    Args:
        origin (list): list of length 2 
        point_set (list): list of shape (N,2)

    Returns:
        tuple: a tuple containing the closest point and the farthest point.
    """    
    distance_metric = lambda x: (x[0] - origin[0])**2 + (x[1] - origin[1])**2 # no need to sqrt for comparing distances
    closest = min(point_set, key=distance_metric)
    farthest = max(point_set, key=distance_metric)

    return closest, farthest 

In [ ]:
# def weighted_mass_gradient(binary_mask:np.ndarray):
#     out_arr = np.zeros(binary_mask.shape)
#     coords_set = np.array(np.nonzero(binary_mask)).T
#     coords_set_complement = np.array(np.nonzero(np.logical_not(binary_mask))).T
    
#     for o in tqdm(coords_set_complement):
        
#         shifted = coords_set - o
#         out_arr[o[0]][o[1]] = sum(1/ np.linalg.norm(shifted, axis=1) ** 2)
    
#     return out_arr


In [ ]:
def windowed_weighted_mass_gradient(binary_mask:np.ndarray, w=20, breadth=2):
    out_arr = np.zeros(binary_mask.shape)
    coords_set_complement = np.array(np.nonzero(np.logical_not(binary_mask))).T

    padded = np.zeros(np.add(binary_mask.shape, w*2))
    padded[w:-w, w:-w] = binary_mask
    
    for i,o in tqdm(enumerate(coords_set_complement)):
        row_min = o[0]
        row_max = o[0] + 2 * w
        col_min = o[1]
        col_max = o[1] + 2 * w

        windowed_coords_set = padded[row_min:row_max, col_min:col_max]
        shifted = np.array(np.nonzero(windowed_coords_set)).T - w
        out_arr[o[0]][o[1]] = sum(1/ np.linalg.norm(shifted, axis=1) ** breadth)# / windowed_coords_set.size

    return out_arr + binary_mask * out_arr.max()

In [ ]:
def adjust_starting_line(b, l, angle):
    """Adjust the location of each point until it is touching tissue. 

    Args:
        b (ndarray): the binary mask
        l (ndarray): the coordinates of L 
        angle (float): the angle of orientation of the tissue.

    Returns:
        ndarray: the new coordinates of the starting line.
    """    
    new_l = []

    # calculate the movement vector from the angle in row col space
    movement_vect = np.array([np.sin(-1 * angle), np.cos(angle)])

    def check_bounds(point, b):   
        return point[0] >= 0 and point[1] >= 0 and point[0] < b.shape[0] and point[1] < b.shape[1]
    
    for point in l:
        hit = False
        n = np.copy(point)
        p = np.copy(point)

        # if the point is already touching tissue, add it to the new list.
        while not hit and (check_bounds(n, b) and check_bounds(p, b)):    # check if the point is within the bounds of the image
            if b[int(n[0]), int(n[1])]:
                new_l.append(n)
                hit = True
            elif b[int(p[0]), int(p[1])]:
                new_l.append(p)
                hit = True
            else:
                n -= movement_vect * 3
                p += movement_vect * 3

        # if not hit:
        #     new_l.append(point)
            
    return np.array(new_l)

In [ ]:
# import concurrent
# from more_itertools import grouper
# def weighted_mass_gradient_concurrent(binary_mask, num_workers, group_size=100):
#     out_arr = np.zeros(binary_mask.shape)
#     coords_set = np.array(np.nonzero(binary_mask)).T
#     coords_set_complement = np.array(np.nonzero(np.logical_not(binary_mask))).T

#     def main_loop(coords_set_complement):
#         for origin in coords_set_complement:
#             shifted = coords_set - origin
#             force = sum(1/np.linalg.norm(shifted, axis=1) ** 2)
#             out_arr[origin[0], origin[1]] = force

#     executor = concurrent.futures.ProcessPoolExecutor(num_workers)
#     groups = grouper(coords_set_complement, group_size)
#     futures = [executor.submit(main_loop, group) for group in groups]
#     concurrent.futures.wait(futures)

#     return out_arr


In [ ]:
# def weighted_mass_gradient(binary_mask:np.ndarray, pbar=None):
#     if pbar is not None:
#         pbar.update(1)
    
#     coords_set = np.array(np.nonzero(binary_mask)).T
#     inv_coords_set = np.array(np.nonzero(np.logical_not(binary_mask))).T
    
#     def inverse_sum_over_coords_vect(origin):
#         c = coords_set - origin
#         return sum(1/(np.linalg.norm(c, axis=1)**2))

#     inverse_sums = np.vectorize(inverse_sum_over_coords_vect, signature='(m,n)->(m)')
    
#     return inverse_sums(inv_coords_set)


In [ ]:
image = Image.new('1', (100, 100))
draw = ImageDraw.Draw(image)
draw.ellipse((20, 20, 60, 60), fill = 'white', outline ='white')
i = np.array(image)
plt.imshow(i)

In [ ]:
im = windowed_weighted_mass_gradient(i, 80, breadth=1)
im2 = windowed_weighted_mass_gradient(i, 80, breadth=1.5)
im3 = windowed_weighted_mass_gradient(i, 80, breadth=2)
im4 = windowed_weighted_mass_gradient(i, 80, breadth=0.2)


In [ ]:
plt.imshow(im)

In [ ]:
plt.plot(im[50])

In [ ]:

fn = '/home/jackson/data/tortuosity_data/_1687394.svs'

fname = fn.split('/')[-1]
osh = openslide.OpenSlide(fn)
print(f'Working on file: {fn}')
print(f'Level dimensions: {osh.level_dimensions}')

img = osh.read_region((0, 0), 2, osh.level_dimensions[2])
basewidth = 1000
wpercent = (basewidth/float(img.size[0]))
hsize = int((float(img.size[1])*float(wpercent)))
img = img.resize((basewidth, hsize))
b = np.asarray(img)[:, :, 0] < 200

b = remove_small_holes(b, 1000)
b = remove_small_objects(b, 100)

# find contours using a gently dilated binary mask.
contours = find_contours(binary_fill_holes(
    binary_dilation(b, iterations=2)))
b_gap_count = len(contours) - 1

# aggressive binary dilation to close gaps
b = binary_dilation(b, iterations=3)
# b = binary_erosion(b, footprint=np.ones((9, 9)))


# compute the distance transform and normalize to the range [0, 0.5]
eb = distance_transform_edt(b)
eb = cv2.normalize(eb, None, 0, 1, cv2.NORM_MINMAX) / 2

# perform a similar computation on the inverted mask. Negate and normalize to the same range.
e_inv_b = distance_transform_edt(np.logical_not(b))
e_inv_b = (1 - cv2.normalize(e_inv_b, None, 0, 1, cv2.NORM_MINMAX)) / 2

# physics-based alternative
# e_inv_b = windowed_weighted_mass_gradient(b, 40, 0.5)
# e_inv_b = (cv2.normalize(e_inv_b, None, 0, 1, cv2.NORM_MINMAX)) / 2


# sum the two distributions
eb_total = eb + e_inv_b


In [ ]:

plt.imshow(eb_total)
# x = np.arange(eb_total.shape[1])
# y = -1.49 * x + 1362
# y = np.array(y, dtype=int)
# y = np.clip(y, 0, eb_total.shape[0] - 1)
# plt.plot(x,y)


$F=\sum \frac{GMm}{r^2} \longrightarrow F=\sum \frac{m}{r^2}$

In [ ]:
plt.plot(eb_total[y, x][500:700], label="eb + eb inverse")
plt.plot(eb[y, x][500:700], label="eb")
plt.plot(e_inv_b[y, x][500:700], label="eb inverse")
plt.legend()

In [ ]:

rp = regionprops(b.astype(np.uint8))[0]
print(rp.orientation)
if rp.orientation <= 0:
    rc_start = (rp.bbox[0], rp.bbox[3])
else:
    rc_start = (rp.bbox[0], rp.bbox[1])

contour_points = []
for contour in contours:
    contour_points.extend(contour)

closest, farthest = closest_farthest_points(rc_start, contour_points)

r = np.linspace(closest[0], farthest[0], 200)
c = np.linspace(closest[1], farthest[1], 200)

init = np.array([r, c]).T


In [ ]:
init_adjusted = adjust_starting_line(b, init, rp.orientation)


In [ ]:
plt.plot(init_adjusted[:, 1], init_adjusted[:, 0], '.b', lw=1, label='L_0')

In [ ]:

print(f'Region Props bounding box: {rp.bbox}')
snake = active_contour(eb_total, init_adjusted, alpha=0.001, beta=.3, gamma=0.001, w_line=50, max_px_move=1,
                    w_edge=0, max_num_iter=1000, convergence=.01, boundary_condition='fixed')

# snake2 = active_contour(eb, snake, alpha=.1, beta=.1, gamma=0.001, w_line=10, max_px_move=1,
#                     w_edge=0, max_num_iter=1000, convergence=.005, boundary_condition='fixed')


# calculate tortuosity in the xy plane
L = np.sum(np.linalg.norm(np.diff(snake, axis=0), axis=-1))
L_0 = np.linalg.norm(init[-1] - init[0])
xy_tortuosity = round(L / L_0, 4)
print(f'xy_tortuosity: {xy_tortuosity}')

# create plot
plt.imshow(img)
fmts = ['c', 'm', 'y', 'k', '#e875fa', '#b5d9ff', '#ffb5bb', '#ffa530', '#3e754d', '#0014c7', '#918211']
print(f'# of contours: {len(contours)}')
for i, c in enumerate(contours):
    plt.plot(c.T[1], c.T[0], f'-.',
                c=f'{fmts[i]}', lw=1, label=f'contour {i}')

plt.plot(snake[:, 1], snake[:, 0], '.r', lw=1, label='L')
plt.plot(init_adjusted[:, 1], init_adjusted[:, 0], '.b', lw=1, label='L_0')
# plt.plot(snake2[:, 1], snake2[:, 0], 'g', lw=1, label='L')
plt.plot(init[:, 1], init[:, 0], '--b', lw=1, label='L_0')
plt.legend()
plt.title(
    f'Filename: {fname}\nTortuosity (L/L_0): {xy_tortuosity}\nNumber of gaps: {b_gap_count}')